- Append(default)--writes the new arrived data      
- Completes--writes entore results every time
- update---updates the recors id the data matches in table and file, writes only new rows that have changed since last micro batch
- option(maxFilesPerTrigger,1)--to be written in read----even though we have multiple files only 1 files is read at a time

In [0]:
from pyspark.sql.types import *
customer_schema_update = StructType(
    fields = [
        StructField('customer_id',IntegerType()),
        StructField('customer_name',StringType()),
        StructField('date_of_birth',DateType()),
        StructField('telephone',StringType()),
        StructField('email',StringType()),
        StructField('member_since',DateType()),
        StructField('created_timestamp',TimestampType()),

    ]
)

In [0]:
customer_df_update = (spark.readStream.format('json')
              .schema(customer_schema_update)
              .load("/Volumes/gizmobox/landing/operational_data/customers_stream/")

)

In [0]:
from pyspark.sql.functions import col, current_timestamp, row_number, split
customer_transform_df_update = (
    customer_df_update
    .withColumn("fname", split(col("customer_name"), " ").getItem(0))
    .withColumn("lname", split(col("customer_name"), " ").getItem(1))
    .withColumn("created_time", current_timestamp())
    .filter(col("customer_id").isNotNull())
)


In [0]:
# for ouput mode update--delta format will not work
#complete only works if the data is aggregrated
stream_query_updated = (customer_transform_df_update.writeStream
                .format("delta")
                .outputMode("append")
                .option("checkpointLocation","/Volumes/gizmobox/landing/operational_data/customers_stream/checkpoint_stream_update")
                .toTable("gizmobox.bronze.customer_stream_update")
                
                
                )